# Data Mining and Statistics
## Session 5 - Classification
*Peter Stikker - Haarlem, the Netherlands - v1.1*

----

In [ ]:
# numpy as np
try:
    import numpy as np
    print('NumPy already installed, only imported')
except:
    !pip install numpy
    import numpy as np
    print('NumPy was not installed, installed and imported')
      
# pyplot as plt
try:
    import matplotlib.pyplot as plt
    print('PyPlot already installed, only imported')
except:
    !pip install matplotlib
    import matplotlib.pyplot as plt
    print('PyPlot was not installed, installed and imported')

# pandas as pd   
try:
    import pandas as pd
    print('pandas already installed, only imported')
except:
    !pip install pandas
    import pandas as pd
    print('pandas was not installed, installed and imported')    



# 5.1. Naive Bayes

Naive Bayes is named after Bayes Theorem, which is:

\begin{equation*}
P(A|B) = \frac{P(B|A)\times P(A)}{P(B)}
\end{equation*}

The $P(A|B)$ needs to be read as 'the probability of A, given that B has occured'. Bayes Theorem makes it possible to calculate this if we have the probability of B, given A ($P(B|A)$).

On the slides we'll do a quick manual calculation, so lets focus here on applying the Naive Bayes.

You might be wondering why it's called 'Naive'. Thats because it assumes that the indpendent variables do not depent on each other, which is termed 'naive'.

### 5.1.1. Naive Bayes with Categorical Input

Most explanations on how Naive Bayes works, is done by using an example with Categorical data. Oddly enough most examples that show applications in Python, work with numerical data. Sklearn has not yet added an option for dealing with categorical data in their stable release, but there is luckily one in beta stage.

We begin with loading in some example data. This was based on the data at https://medium.com/@prgopinath100/naive-bayes-classifier-dd8b179b5d90. 

In [ ]:
# variable name 'seldf' as in 'selection data frame'
seldf=pd.read_csv('data/selection.csv', sep = ';', names=["experience", "form", "fitness", "selected"])
seldf

To use the sklearn Naive Bayes option for Categorical input we unfortunately this will need to be re-coded into numerical values. We first let panda know which fields are 'Categorical':

In [ ]:
seldf["experience"]=pd.Categorical(seldf["experience"])
seldf["form"]=pd.Categorical(seldf["form"])
seldf["fitness"]=pd.Categorical(seldf["fitness"])
seldf["selected"]=pd.Categorical(seldf["selected"])

Now for each field the different categories can be obained using cat.code:

In [ ]:
seldf["experience"].cat.categories

And the numeric values of the field can be shown using .cat.codes:

In [ ]:
seldf["experience"].cat.codes

We need the numeric codes, so we build a numpy array using those codes:

In [ ]:
# The variable name 'selarr' was chosen, short for 'selection array'
selarr = np.asarray(seldf["experience"].cat.codes)
selarr = np.dstack((selarr, np.asarray(seldf["form"].cat.codes)))
selarr = np.dstack((selarr, np.asarray(seldf["fitness"].cat.codes)))
selarr = np.dstack((selarr, np.asarray(seldf["selected"].cat.codes)))
selarr = np.squeeze(selarr)
selarr

Finally we can build our model.

Define our predictors, and what we want to predict:

In [ ]:
X=selarr[:,0:3]     ##the experience, form, and fitness
y=selarr[:,3]       ##the selection

Now we can use the CategoricalNB from sklearn:

In [ ]:
from sklearn.naive_bayes import CategoricalNB

Now, let's see how this performs:

In [ ]:
catNB = CategoricalNB()
catNB.fit(X, y)
print(catNB.predict(X))

So these are the results of the analysis for the 'selected'. As a reminder we can find out what a 0 or 1 means:

In [ ]:
seldf["selected"].cat.categories

So a 0 = no and 1 = yes. The earlier found [1 1 0 1 0 1 1 1 0 1 0 0] therefor indicates that the first and second person is predicted to be selected, the third isn't, the fourth is, etc.

To see how well the model performed:

In [ ]:
catNB.score(X,y)

The model was correct is 64% of the cases. It was wrong for person 7, 8, 9 and 10, so correct in 7 out of 11 cases, which indeed equals 64%.

And to actually see the probabilities:

In [ ]:
catNB.predict_proba(X)

The first person is predicted to have a 44% chance of not being selected, and a 56% chance of being selected. Since 56 > 44, we'd predict this person to be selected.

Lets do a prediction for someone whos experience = 1, form = 1 and Fitness = 1:

In [ ]:
myTest=[1,1,1]
myTest=np.array(myTest)
myTest = myTest.reshape(1, -1)
print(catNB.predict(myTest))

The predicted result is a 1, so a good chance to be selected.

**Exercise 1**

Determine who is most likely to win in a match between a Southpaw fighter and a Orthodox player.

Use the UFC2019.csv dataset and of course a naive Bayesian analysis.

In [ ]:
# Load the data

# Show first few rows


In [ ]:
# Create subset of data and remove missing values


# Convert panda fields to categorical


# get the numerical values as a numpy array


# set the independent (X) and dependent variable (Y)


# Create and fit the model


# Show some results



**Exercise 2**

Another example taken from: https://www.saedsayad.com/naive_bayesian.htm. The data is already available as 'playGolf.csv'. Load this data and create a model to predict if we can go Play or not.

If you have time to spare, you could look into the conversion of the categories into the numerical ones by using the LabelEncoder option of sklearn.

In [ ]:
# Load the data


# Show first few rows


In [ ]:
# Convert panda fields to categorical

# get the numerical values as a numpy array


# set the independent (X) and dependent variable (Y)


# Create and fit the model


# Show some results


### 5.1.2. Naive Bayes with numerical data

So, now that we've seen how to create a Naive Bayes model, using categorical input variables. Lets go to the more commonly used version, with numerical data. 

With numerical data we usually cannot do a simple frequency count (many scores would have a frequency of 1 or just a few). For these it is often assumed they will follow a Normal Distribution, a.k.a. Gaussian Distribution. We won't go into the details of this and leave the calculations up to Python.

First again some example data:

In [ ]:
# Load original UFC data
UFCdata=pd.read_csv('data/UFC2019.csv',sep = ',', header=0)
# store field names in separate list
UFCfields=list(UFCdata.columns)

#Convert numerical fields into own dataframe.
UFCnum=pd.DataFrame()

for i in UFCfields:
    if (UFCdata[i].dtype=='int64' or UFCdata[i].dtype=='float64'):
        UFCnum[i]=UFCdata[i]

#Add the winner
UFCnum['Winner']=UFCdata['Winner']

Now to make the model and look at the prediction:

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix

# Import the GaussianNB so we can use the commonly used Gaussian transformation
from sklearn.naive_bayes import GaussianNB

# Set our model to this GaussianNB
gaussNB = GaussianNB()

#Get the specific fields of interest
mDat =UFCnum[["B_age", "R_age", "Winner"]]
# remove missing records and draws
mDat=mDat.dropna()
mDat = mDat[mDat.Winner != 'Draw']

# Set input and output
X = mDat[["B_age", "R_age"]]
y = mDat["Winner"]

#Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=109)


#Create the model and prediction
gaussNB.fit(X_train, y_train)
y_pred = gaussNB.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

Our model had 66% correct in the testing data.

We can have a more detailed look, by looking at the so-called confusion matrix:

In [ ]:
print(confusion_matrix(y_test, y_pred))

The top left value of 19, indicates that our model predicted a blue and it actually also was a blue 19 times.

The bottom right value of 626 indicates that our model predicted a red and it actually also was a red 626 times.

In the other cases it was wrong (311 times predicted red, but was blue, and 18 times predicted blue and was red).

To double-check the math:

In [ ]:
(19+626)/(19+311+18+626)

The same 66% as we saw earlier.

We can see that it did try its best:

In [ ]:
gaussNB.predict_proba(X)

### 5.1.3. Only binary outcomes?
What if our outcome variable is not two, but three or more categories?

Well lets see what happens if we don't remove the draws.

In [ ]:
# The original data
mDat = UFCnum[["B_age", "R_age", "Winner"]]
winTypes ={"Red":0, "Blue":1, "Draw":2} 

# Convert 'Winner' to numerical values as a list
numWin = [winTypes[i] for i in mDat["Winner"]]  
numWin = pd.DataFrame(numWin)

# Combine the two data frames
mDat = pd.concat([mDat,numWin], axis=1, sort=False)

# Remove the original 'Winner'
mDat = mDat.drop(["Winner"], axis=1)

# Rename the columns
mDat.columns = ["B_age", "R_age", "Winner"]

#Remove the missing values
mDat=mDat.dropna()

# Now get X and Y:
X = mDat[["B_age", "R_age"]]
y = mDat["Winner"]

# And convert them to a numpy array
X = np.asarray(X)
y = np.asarray(y)

#Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=109)

#Create the model and prediction
gaussNB.fit(X_train, y_train)
y_pred = gaussNB.predict(X_test)

# Finally the result:
print(metrics.accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Well, there doesn't seem to be any problem. 

It did not predict any draws, but we only have 19 draws.

In [ ]:
np.unique(y_test, return_counts=True)

**Exercise 3**

Improve the model by adding some other variables you think might be useful...

### 5.1.4. Other similar techniques
We discussed the basic ideas and concepts of the Naive Bayes method for classification. There is also 

* Multinomial Naive Bayes <br>
Helpful if your numerical values are counts.
* Complement Naive Bayes <br>
an extension of Multinomial Naive Bayes and used if the dataset is imbalanced.
* Bernoulli Naive Bayes <br>
if you have binary/boolean features.
* Average One-Dependence Estimators (AODE), which is a bit out of scope from this course.

Before we finish this chapter. Lets take a quick peek at our original data and the winners (should we have done this when we started?)

In [ ]:
UFCdata["Winner"].value_counts()

Hm. Only 83 draws and Red seems to win most of the time. Is there a reason why this might happen? We would need an expert on MMA to find this out. Perhaps if we see Micha he can explain. 

Alright, perhaps another technique can improve our performance.

<center><b>BACK TO THE SLIDES</b></center>

## 5.2. Support Vector Machines

Sklearn has a nice Python script themselves to be used as an example (https://scikit-learn.org/stable/auto_examples/svm/plot_separating_hyperplane.html) and also on https://jakevdp.github.io/PythonDataScienceHandbook/05.07-support-vector-machines.html some very nice explanations are given. So lets not re-invent the wheel, and I've used some of their bits.

Beginning with creating some data.

In [ ]:
from sklearn.datasets import make_blobs
X, y = make_blobs(n_samples=50, centers=2,
                  random_state=18, cluster_std=1.60)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50);

Now to create our Support Vector Machine (or Support Vector Classifier). 

We first use the linear kernel:

In [ ]:
# Import the required SVC package from sklearn
from sklearn.svm import SVC

# Create the model, then fit the model, and determine the predicted values
svmLin = SVC(kernel='linear')
svmLin.fit(X, y)
y_pred = svmLin.predict(X)

# Finally the result:
print(metrics.accuracy_score(y, y_pred))
print(confusion_matrix(y, y_pred))

We can also show the result, using the small function below:

In [ ]:
def showHyperplane(X, y, model):
    plt.scatter(X[:, 0], X[:, 1], c=y, s=30, cmap=plt.cm.Paired)

    # plot the decision function
    ax = plt.gca()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()

    # create grid to evaluate model
    xx = np.linspace(xlim[0], xlim[1], 30)
    yy = np.linspace(ylim[0], ylim[1], 30)
    YY, XX = np.meshgrid(yy, xx)
    xy = np.vstack([XX.ravel(), YY.ravel()]).T
    Z = svmLin.decision_function(xy).reshape(XX.shape)

    # plot decision boundary and margins
    ax.contour(XX, YY, Z, colors='k', levels=[-1, 0, 1], alpha=0.5,
               linestyles=['--', '-', '--'])
    # plot support vectors
    ax.scatter(svmLin.support_vectors_[:, 0], svmLin.support_vectors_[:, 1], s=100,
               linewidth=1, facecolors='none', edgecolors='k')
    plt.show()

In [ ]:
showHyperplane(X, y, svmLin)

The other example on the slides was the circular data.

In [ ]:
from sklearn.datasets import make_circles
X, y = make_circles(100, factor=.1, noise=.1)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')

Oh, oh. Doesn't look promising to draw a straight line to separate the red from yellow dots. Lets see how well sklearn can do this anyway:

In [ ]:
svmLin = SVC(kernel='linear').fit(X, y)
svmLin.score(X,y)

Hm, still 69% correct. What would this look like:

In [ ]:
showHyperplane(X,y,svmLin)

Well, its something but by observation alone we can see that the hyperplane should not be a straight line anymore, but a circle is much more obvious.

Translate our scores to circle-style:

In [ ]:
Z=X[:,0]**2+X[:,1]**2

Visualize using ipywidgets, so make sure you have installed it (using pip install ipywidgets)

In [ ]:
from ipywidgets import interact, fixed

def plot_3D(elev=30, azim=30, X=X, y=y):
    ax = plt.subplot(projection='3d')
    ax.scatter3D(X[:, 0], X[:, 1], Z, c=y, s=50, cmap='autumn')
    ax.view_init(elev=elev, azim=azim)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('Z')

interact(plot_3D, elev=[20, 70], azip=(-180, 180),
         X=fixed(X), y=fixed(y));

Find the plane that separates these points:

In [ ]:
svmRBF = SVC(kernel='rbf').fit(X, y)
y_pred = svmRBF.predict(X)

# The result:
print(metrics.accuracy_score(y, y_pred))
print(confusion_matrix(y, y_pred))

Great. This separates all our points correctly (we should of course use some test and training data, but you get the point)

Lets apply this to our UFC data. First just in case select the data again, and split the data.

In [ ]:
#Make a subset of the data and remove missing values
UFCsel = UFCdata[["Winner", "B_age", "R_age"]]
UFCsel=UFCsel.dropna()

#Determine input and output variables
X = UFCsel[["B_age", "R_age"]]
y = UFCsel["Winner"]

#Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=109)

Now to create the models:

In [ ]:

# SVM with linear kernel
svmLin = SVC(kernel='linear')
svmLin.fit(X_train, y_train)
y_pred = svmLin.predict(X_test)

print('SVM Linear Kernel results')
print('accuracy score: ', metrics.accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

print('')

# SVM with Radial Basis Function kernel
rbfKer = SVC(kernel='rbf')
rbfKer.fit(X_train, y_train)
y_pred = rbfKer.predict(X_test)

print('SVM RBF Kernel results')
print('accuracy score: ', metrics.accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

There some other kernels as well available in sklearn. Unfortunately not enough time to explain all of them, but just FYI:

In [ ]:
#This one takes a long time, so commented it out

## SVM with Polynomial kernel
#svmPol = SVC(kernel='poly')
#svmPol.fit(X_train, y_train)
#y_pred = svmPol.predict(X_test)

#print('accuracy score: ',metrics.accuracy_score(y_test, y_pred))
#print(confusion_matrix(y_test, y_pred))

#Result was 0.6636363636...

In [ ]:
# SVM with Sigmoid Kernel
svmSigm = SVC(kernel='sigmoid')
svmSigm.fit(X_train, y_train)
y_pred = svmSigm.predict(X_test)

print('accuracy score: ', metrics.accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

One thing to definately check is how many support vectors we actually have. If this is too high we might be overfitting the data. Usually at 10% or more you should get a bit worried.

In [ ]:
print('Linear kernel percentage: ',len(svmLin.support_)/len(X_train))
print('RBF kernel percentage: ',len(svmRBF.support_)/len(X_train))
#print('Polynomial kernel percentage: ',len(polKer.support_)/len(X_train))
print('Sigmoid kernel percentage: ',len(svmSigm.support_)/len(X_train))

Ai, thats more than 65% of our test data points are used to create the hyperplane itself, except for the RBF kernel. So at this moment I'd go for that one.

For the other kernels we might need to soften our margin (the C parameter). 

But lets see how another technique might actually perform...

<center><b>BACK TO THE SLIDES</b></center>

## 5.3. KNN Classification

On the slides the basic idea of kNN was discussed. Implementing this with sklearn is very similar as we've seen before:

In [ ]:
# import the KNeighborsClassifier package
from sklearn.neighbors import KNeighborsClassifier

# Setup and fit the model (we use the same data as before)
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# Create the prediction
y_pred=knn.predict(X_test)

# Show the results
print('accuracy score: ', metrics.accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Almost a must in KNN classification is to set the data to a standard scale. This is accomplished by subtracting the mean from each value, and then divide the result by the standard deviation. These are also known as Z-scores. In formula notation:

\begin{equation*}
Z = \frac{x - \bar{x}}{\sigma}
\end{equation*}

The $\bar{x}$ is the mean of field X, and $\sigma$ is the population standard deviation (the sample standard deviation to be a bit more precise)

We could of course easily do this ourselves:

In [ ]:
Zscores = (X_train - X_train.mean())/X_train.std(ddof=0)

But of course, sklearn has us covered...

In [ ]:
# Import the standardscaler package
from sklearn.preprocessing import StandardScaler

# set the scaler
scaler = StandardScaler()
scaler.fit(X_train)

# Convert the train and test X values, using the same scaler (so based on the X_train)
X_trainScaled = scaler.transform(X_train)
X_testScaled = scaler.transform(X_test)

Quick check if they are indeed the same:

In [ ]:
X_trainScaled - Zscores

Lets see if this pays off.

In [ ]:
knn.fit(X_trainScaled, y_train)
y_pred=knn.predict(X_testScaled)
print(metrics.accuracy_score(y_test, y_pred))

Changed a bit, but not much.

One suggestion for the setting of k (the number of neighbors to consider) is to take the square root of the number of data points:

In [ ]:
X_train.shape[0]**0.5

An odd number is usually preferred (to avoid ties), so lets use 63 and see if that helps:

In [ ]:
knn = KNeighborsClassifier(n_neighbors=63)

knn.fit(X_trainScaled, y_train)
y_pred=knn.predict(X_testScaled)
print(metrics.accuracy_score(y_test, y_pred))

That's about 3% better. We can also just iterate over a range and look which performs best:

In [ ]:
# Thanks for the script: https://datascienceplus.com/k-nearest-neighbors-knn-with-python/
error_rate=[]
for i in range(1,80):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_trainScaled, y_train)
    pred_i = knn.predict(X_testScaled)
    error_rate.append(np.mean(pred_i != y_test))

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(range(1,80),error_rate,color='blue', linestyle='dashed', marker='o',
         markerfacecolor='red', markersize=10)
plt.title('Error Rate vs. K Value')
plt.xlabel('K')
plt.ylabel('Error Rate')

At 43 things go slightly up again and not much differences anymore after that. Lets check it out:

In [ ]:
knn = KNeighborsClassifier(n_neighbors=43)

knn.fit(X_trainScaled, y_train)
y_pred=knn.predict(X_testScaled)
print(metrics.accuracy_score(y_test, y_pred))

Hm, only another 1% increase in our performance, but less computational than the k=63

Another approach could be to use the so called 'GridSearchCV' package, which will iterate over a lot of settings.

In [ ]:
from sklearn.model_selection import GridSearchCV

The code below was copied from https://medium.com/datadriveninvestor/k-nearest-neighbors-in-python-hyperparameters-tuning-716734bc557f

I've commented out the code since it takes a very long time.

In [ ]:
## List Hyperparameters that we want to tune.
#leaf_size = list(range(1,50))
#n_neighbors = list(range(1,30))
#p=[1,2]
##Convert to dictionary
#hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)
##Create new KNN object
#knn_2 = KNeighborsClassifier()
##Use GridSearch
#clf = GridSearchCV(knn_2, hyperparameters, cv=10)
##Fit the model
#best_model = clf.fit(X_trainScaled, y_train)
##Print The value of best Hyperparameters
#print('Best leaf_size:', best_model.best_estimator_.get_params()['leaf_size'])
#print('Best p:', best_model.best_estimator_.get_params()['p'])
#print('Best n_neighbors:', best_model.best_estimator_.get_params()['n_neighbors'])

# The result was a leaf size of 1, p of 1 and 29 neighbors

Using the results:

In [ ]:
knnOptimal = KNeighborsClassifier(n_neighbors=29, leaf_size=1, p=1)
y_pred=knn.predict(X_testScaled)
print(metrics.accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Similar result, but a lot lower number of neighbors.

Our model never seems to predict a 'Draw'. One possible reason might be that we are looking at the 29 neighbors of a point, and we only have 17 draws in total. The chances that a draw will win a majority vote is very low.

If you find the confusion matrix, well confusing. You might also use the classification report:

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

I'll leave it up to you to google on how to interpret this, since we still have a lot to discuss. There are so many more techniques.

The optimal settings from GridSearch set something called 'leafs'. To understand this better it can help to look at our next technique: Decision Trees.....

<center><b>BACK TO THE SLIDES</b></center>

## 5.3. Decision trees

The concept of Decision trees is explained on the slide. The simplest version requires another package, so **first run 'pip install decision-tree-id3' in shell**, then come back here to actually load the package

In [ ]:
# !pip install decision-tree-id3

# importing it directly seems to have a small bug, so
# first import six and sys
import six
import sys
sys.modules['sklearn.externals.six'] = six
# then finally id3
from id3 import Id3Estimator

In [ ]:
id3_dtc=Id3Estimator()
id3_dtc.fit(X_trainScaled,y_train)
y_pred = id3_dtc.predict(X_testScaled)
print(metrics.accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Not bad.

sklearn also has a 'DecisionTreeClassifier' and we can set the criterion for that to 'entropy':

In [ ]:
from sklearn.tree import DecisionTreeClassifier 

ent_dtc = DecisionTreeClassifier(criterion = "entropy")
ent_dtc.fit(X_trainScaled,y_train)
y_pred = ent_dtc.predict(X_testScaled)
print(metrics.accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Hmm. The documentation of DecisionTreeClassifier mentions: "...“entropy” for the information gain.", so this should be the same as the id3. It could be that either there is a small variation in the id3 method, or sklearn uses different default settings than id3. Something to look into.

Sklearn does not have a pure C4.5 implementation, but usually CART is enough. We can get this by either not setting a criterion (since it's the default), or set it to 'gini':

In [ ]:
gini_dtc = DecisionTreeClassifier(criterion = "gini")
gini_dtc.fit(X_trainScaled,y_train)
y_pred = gini_dtc.predict(X_testScaled)
print(metrics.accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

The random forest classifier is also available in sklearn:

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=0)
rfcModel = rfc.fit(X_trainScaled, y_train)
y_pred = rfcModel.predict(X_testScaled)
print(metrics.accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Just because its there, a quick show of the Extra Trees classifier:

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

# Extremly Random Forest (a.k.a. Extra trees)
erfc = ExtraTreesClassifier(random_state=0)
erfc = erfc.fit(X_trainScaled, y_train)
y_pred = erfc.predict(X_testScaled)
print(metrics.accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

The Random Forest and Extra Trees seem to outperform the best decision tree. Actually Random Forest and Extra Trees are not really a decision tree, but more a decision tree**s**. This is because it is a bagging (bootstrap aggregation) technique. What's that you ask? Well....

<center><b>BACK TO THE SLIDES</b></center>

## 5.4. And Many More

In this section just some basic code showing a few more methods.

**Linear Discriminant Analysis**

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda = LinearDiscriminantAnalysis()
ldaModel=lda.fit(X_trainScaled, y_train)
y_pred=ldaModel.predict(X_testScaled)
print(metrics.accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

**Quadratic Discriminant Analysis**

In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

qda = QuadraticDiscriminantAnalysis()
qdaModel=qda.fit(X_trainScaled, y_train)
y_pred=ldaModel.predict(X_testScaled)
print(metrics.accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

**Logistic Regression Classification**

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
lrModel = logreg.fit(X_trainScaled, y_train)
y_pred = lrModel.predict(X_testScaled)
print(metrics.accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

**Multinomial Logistic Regression Classification**

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(multi_class='multinomial')
lrModel = logreg.fit(X_trainScaled, y_train)
y_pred = lrModel.predict(X_testScaled)
print(metrics.accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

**AdaBoos**
(Adaptive Boosting)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

adaBst = AdaBoostClassifier(random_state=0)
adaBst = adaBst.fit(X_trainScaled, y_train)
y_pred = adaBst.predict(X_testScaled)
print(metrics.accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

**Gradient Boosting**

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gradBst = GradientBoostingClassifier(random_state=0)
gradBst = gradBst.fit(X_trainScaled, y_train)
y_pred = gradBst.predict(X_testScaled)
print(metrics.accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

**Histogram Gradient Boosting**

In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

histBst = HistGradientBoostingClassifier(random_state=0)
histBst = histBst.fit(X_trainScaled, y_train)
y_pred = histBst.predict(X_testScaled)
print(metrics.accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

**Stacking**

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC

estimators = [
    ('rf', RandomForestClassifier(n_estimators=10, random_state=42)),
    ('svr', make_pipeline(StandardScaler(),
                          LinearSVC(random_state=42)))]

stackCl = StackingClassifier(estimators=estimators, final_estimator = LogisticRegression())
stackCl.fit(X_trainScaled, y_train)
y_pred = stackCl.predict(X_testScaled)
print(metrics.accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

**Dummy Classifier - Most Frequent** <br>
Sometimes referred to as ZeroR classification.

In [ ]:
from sklearn.dummy import DummyClassifier

dumMF = DummyClassifier(strategy='most_frequent')
dumMF = dumMF.fit(X_trainScaled, y_train)
y_pred = dumMF.predict(X_testScaled)
print(metrics.accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

**Dummy Classifier - Stratified**

In [ ]:
# Dummy classifier with stratified method
dumSrat = DummyClassifier(strategy="stratified")
dumSrat = dumSrat.fit(X_trainScaled, y_train)
y_pred = dumSrat.predict(X_testScaled)
print(metrics.accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

**Dummy Classifier - Prior**

In [ ]:
dumPrior = DummyClassifier(strategy="prior")
dumPrior.fit(X_trainScaled, y_train)
y_pred = dumPrior.predict(X_testScaled)
print(metrics.accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

**Dummy Clasifier - Uniform**

In [ ]:
dumUni = DummyClassifier(strategy="uniform")
dumUni.fit(X_trainScaled, y_train)
y_pred = dumUni.predict(X_testScaled)
print(metrics.accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

<center><b>BACK TO THE SLIDES</b></center>

# Encore

After all these different models, I was a bit curious to just list all of them in one go. So...

In [ ]:
# Create a panda dataframe to keep track of the scores
accScores = pd.DataFrame(columns = ['model', 'score'])

# Create a list with all the different models (except polynomial svm).
models =[["Dummy - Uniform", DummyClassifier(strategy="uniform")]]
models.append(['Dummy - Most Freq.', DummyClassifier(strategy="stratified")])
models.append(['Dummy - Prior', DummyClassifier(strategy="prior")])
models.append(['Dummy - Most Freq.', DummyClassifier(strategy='most_frequent')])
models.append(['NB - Gaussian', GaussianNB()])
models.append(['SVM - Gaussian', SVC(kernel='linear')])
models.append(['SVM - RBF', SVC(kernel='rbf')])
models.append(['SVM - Sigmoid', SVC(kernel='sigmoid')])
models.append(['kNN', KNeighborsClassifier(n_neighbors=29, leaf_size=1, p=1)])
models.append(['DT - ID3', Id3Estimator()])
models.append(['DT - ID3 (sklearn)', DecisionTreeClassifier(criterion = "entropy")])
models.append(['DT - CART', DecisionTreeClassifier(criterion = "gini")])
models.append(['Bagging - Random Forest', RandomForestClassifier(random_state=0)])
models.append(['Bagging - Extra Trees', ExtraTreesClassifier(random_state=0)])
models.append(['LDA', LinearDiscriminantAnalysis()])
models.append(['QDA', QuadraticDiscriminantAnalysis()])
models.append(['Logistic Regression', LogisticRegression()])
models.append(['Multinomial LR', LogisticRegression(multi_class='multinomial')])
models.append(['Boosting - AdaBoost', AdaBoostClassifier(random_state=0)])
models.append(['Boosting - Gradient', GradientBoostingClassifier(random_state=0)])
models.append(['Boosting - Histogram Gradient', HistGradientBoostingClassifier(random_state=0)])
estimators = [
    ('rf', RandomForestClassifier(n_estimators=10, random_state=42)),
    ('svr', make_pipeline(StandardScaler(),
                          LinearSVC(random_state=42)))]
models.append(['Stacking', StackingClassifier(estimators=estimators, final_estimator = LogisticRegression())])

In [ ]:
# Now to iterate over all of them:
for i in models:
    model = i[1]
    model.fit(X_trainScaled, y_train)
    y_pred = model.predict(X_testScaled)
    score=metrics.accuracy_score(y_test, y_pred)
    newRow=pd.Series([i[0], score], index=accScores.columns)
    accScores=accScores.append(newRow, ignore_index=True)
    
# And show all results sorted by their score:
accScores.sort_values(by=['score'])

There is also XGboost, but that requires numerical data. If you are interested check out https://machinelearningmastery.com/data-preparation-gradient-boosting-xgboost-python/